In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import hankel

import tensorflow as tf 
from tensorflow import keras
from keras.layers import LSTM, Dropout, Dense

2025-01-15 16:59:15.605199: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-15 16:59:15.614341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736949555.626968  744042 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736949555.630148  744042 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 16:59:15.641302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [3]:
from lempel_ziv import *
from huffman import *
from utils import *

In [4]:
data = pd.read_csv('data/transact_18_22.csv', parse_dates=[2])
data.head(3)

,client,card,date,amt,mcc,group,value
0,390989,3048567,2018-01-01,878.0,5814,food,survival
1,1309946,3226559,2018-01-01,249.0,5411,food,survival
2,1386278,2715990,2018-01-01,50.0,5499,food,survival


In [5]:
filtered_data = data[(data['date'] >= '2019-01-15') & (data['date'] <= '2019-12-14')]
filtered_data.head(3)

,client,card,date,amt,mcc,group,value
3278420,1378858,2391344,2019-01-15,350.87,5499,food,survival
3278421,3103680,3187433,2019-01-15,2660.00,5331,nonfood,socialization
3278422,2136426,3762147,2019-01-15,485.00,5331,nonfood,socialization


In [10]:
grouped = filtered_data.groupby(['client', 'date'])
grouped.head()

,client,card,date,amt,mcc,group,value
3278420,1378858,2391344,2019-01-15,350.87,5499,food,survival
3278421,3103680,3187433,2019-01-15,2660.00,5331,nonfood,socialization
3278422,2136426,3762147,2019-01-15,485.00,5331,nonfood,socialization
3278423,2450420,3674008,2019-01-15,4000.00,6011,money,money
3278424,1134488,2862980,2019-01-15,1404.40,5411,food,survival
...,...,...,...,...,...,...,...
6905032,96987,5123908,2019-12-14,6690.00,5999,nonfood,socialization
6905034,1884422,4732444,2019-12-14,1500.00,5732,telecom,socialization
6905035,1414524,4625952,2019-12-14,4000.00,6011,money,money
6905036,2136509,3573708,2019-12-14,97.90,5411,food,survival


In [15]:
values_to_keep = ['food', 'nonfood', 'money', 'outfit', 'travel', 'health', 'fun', 'dwelling', 'beauty', 'telecom', 'kids', 'misc', 'remote', 'charity']
filtered_data = filtered_data[filtered_data['group'].isin(values_to_keep)]

In [16]:
grouped = filtered_data.groupby(['client', 'date'])
new_data = []
for (client, date), group in grouped:
    row = {'client': client, 'date': date}
    for val in values_to_keep:
        row[val] = 1 if val in group['group'].values else 0  
    new_data.append(row)

new_df = pd.DataFrame(new_data)
new_df

,client,date,food,nonfood,money,outfit,travel,health,fun,dwelling,beauty,telecom,kids,misc,remote,charity
0,224,2019-01-15,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,224,2019-01-16,1,0,0,0,1,0,1,0,0,0,0,0,0,0
2,224,2019-01-17,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3,224,2019-01-18,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,224,2019-01-19,0,1,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588462,3564569,2019-12-10,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1588463,3564569,2019-12-11,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1588464,3564569,2019-12-12,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1588465,3564569,2019-12-13,1,0,0,0,0,1,1,0,0,0,0,0,0,0


In [23]:
new_df['code'] = new_df[values_to_keep].apply(binary_to_char, axis=1)
new_df

,client,date,food,nonfood,money,outfit,travel,health,fun,dwelling,beauty,telecom,kids,misc,remote,charity,code
0,224,2019-01-15,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8
1,224,2019-01-16,1,0,0,0,1,0,1,0,0,0,0,0,0,0,c
2,224,2019-01-17,1,0,0,0,1,0,0,0,0,0,0,0,0,0,s
3,224,2019-01-18,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8
4,224,2019-01-19,0,1,0,0,0,0,1,0,0,0,0,0,0,0,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588462,3564569,2019-12-10,1,0,0,0,0,0,1,0,0,0,0,0,0,0,4
1588463,3564569,2019-12-11,1,0,0,0,1,0,0,0,0,0,0,0,0,0,s
1588464,3564569,2019-12-12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,k
1588465,3564569,2019-12-13,1,0,0,0,0,1,1,0,0,0,0,0,0,0,8


In [24]:
clients_counts = new_df.groupby('client').size().reset_index(name='transaction_count')
sorted_clients = clients_counts.sort_values('transaction_count', ascending=False)
clients_counts = sorted_clients.head(1000)
new_df = new_df[new_df['client'].isin(clients_counts['client'])]
new_df

,client,date,food,nonfood,money,outfit,travel,health,fun,dwelling,beauty,telecom,kids,misc,remote,charity,code
0,224,2019-01-15,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8
1,224,2019-01-16,1,0,0,0,1,0,1,0,0,0,0,0,0,0,c
2,224,2019-01-17,1,0,0,0,1,0,0,0,0,0,0,0,0,0,s
3,224,2019-01-18,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8
4,224,2019-01-19,0,1,0,0,0,0,1,0,0,0,0,0,0,0,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588462,3564569,2019-12-10,1,0,0,0,0,0,1,0,0,0,0,0,0,0,4
1588463,3564569,2019-12-11,1,0,0,0,1,0,0,0,0,0,0,0,0,0,s
1588464,3564569,2019-12-12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,k
1588465,3564569,2019-12-13,1,0,0,0,0,1,1,0,0,0,0,0,0,0,8


In [25]:
selected_columns = ['date', 'code'] + values_to_keep

In [26]:
split = 52
lzw_depth = 56
forecast_horizons = [7]
start = 0

In [27]:
clients = list(new_df['client'].value_counts().index)

In [36]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'hp_{category}_{horizon}' for horizon in forecast_horizons for category in values_to_keep] + ['huffman_complexity']+ [f'lz_{category}_complexity' for category in values_to_keep]
)
results_df['client_id'] = clients  
results_df.head(5)

,client_id,hp_food_7,hp_nonfood_7,hp_money_7,hp_outfit_7,hp_travel_7,hp_health_7,hp_fun_7,hp_dwelling_7,hp_beauty_7,...,lz_travel_complexity,lz_health_complexity,lz_fun_complexity,lz_dwelling_complexity,lz_beauty_complexity,lz_telecom_complexity,lz_kids_complexity,lz_misc_complexity,lz_remote_complexity,lz_charity_complexity
0,2190618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,208625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3441340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3483687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1421807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
num_clients = len(clients)
train_days = 180

In [38]:
models = [keras.Sequential([
                LSTM(28, return_sequences=True),
                Dropout(0.3),
                LSTM(28, return_sequences=False),
                Dense(28, activation='tanh'),
                Dense(7, activation='sigmoid')
            ]) for i in range(len(values_to_keep))]

for i in range(len(values_to_keep)):
  models[i].compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
    for i in range(start, start + num_clients):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]

        text = client_transactions['code'].values
        huffman_codes = huffman_code(text)
        encoded_string = ''.join(huffman_codes[symbol] for symbol in text)
        complexity = len(encoded_string) / len(text) / 4.0


        for j, value in enumerate(values_to_keep):
            text = client_transactions[value].values
            lzc = lempel_ziv_complexity(text)

            train_data = text[:train_days]
            test_data = text[train_days:]
            X_train, y_train = create_dataset(train_data, lzc, 7)
            X_test, y_test = create_dataset(test_data, lzc, 7)

            models[j].fit(X_train, y_train, epochs=15, batch_size=1, verbose=0)
            f1_scores, hp_scores = [], []

            for k in range(y_test.shape[0] - 1):
                predictions = models[j].predict(X_test[k:k+1], verbose=0)
                f1, accuracy = calculate_f1_accuracy(predictions.flatten().round(0).astype(int), y_test[k].flatten())
                f1_scores.append(f1)

            if len(f1_scores) > 0:
                hp_score = np.mean(np.array(f1_scores) > 0.75)
            else:
                hp_score = 0.0

            results_df.loc[results_df['client_id'] == clients[i], f'hp_{value}_7'] = hp_score
            results_df.loc[results_df['client_id'] == clients[i], 'huffman_complexity'] = complexity
            results_df.loc[results_df['client_id'] == clients[i], f'lz_{value}_complexity'] = lzc
        if i % 5 == 0:
            results_df.to_csv('group.csv')